In [132]:
import random
import math
import gym
import numpy as np
import collections 
import PIL
from PIL import Image
import matplotlib
import matplotlib.cm as cm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [133]:
class model(nn.Module):
    def __init__(self):
        super(model,self).__init__()
        self.fc1= nn.Linear(8,100)
        self.fc2 = nn.Linear(100, 3)
    
    def forward(self,x):
        x=x.reshape(x.shape[0],1,8)
        x=self.fc1(x)
        return self.fc2(x.contiguous().view(x.size(0), -1))
        

In [134]:
env = gym.make('MountainCar-v0').unwrapped
env.reset()

array([-0.5107029,  0.       ])

In [135]:
policy=model()
target_net=model()
target_net.load_state_dict(policy.state_dict())
target_net.eval()
optimizer = optim.RMSprop(policy.parameters())
criterion = F.smooth_l1_loss
memory=10000
store=[[dict()] for i in range(memory)]
gamma=0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200

In [136]:
def addEpisode(ind,prev,curr,reward,act):
    if len(store[ind]) ==0:
        store[ind][0]={'prev':prev,'curr':curr,'reward':reward,'action':act}
    else:
        store[ind].append({'prev':prev,'curr':curr,'reward':reward,'action':act})

In [137]:
def StackStates(queue):
    stack = np.zeros((8))
    stack[:2]=queue[0]
    stack[2:4]=queue[1]
    stack[4:6]=queue[2]
    stack[6:8]=queue[3]
    return stack
    

In [138]:
def trainNet(total_episodes):
    if total_episodes==0:
        return
    ep=random.randint(0,total_episodes-1)
    if len(store[ep]) < 8:
        return
    else:  
        start=random.randint(1,len(store[ep])-8)
        length=len(store[ep])
        inp=[]
        target=[]
        rew=torch.Tensor(1,8)
        actions=torch.Tensor(1,8)
        
        for i in range(start,start+8,1):
            inp.append((store[ep][i]).get('prev'))
            target.append((store[ep][i]).get('curr'))
            rew[0][i-start]=store[ep][i].get('reward')
            actions[0][i-start]=store[ep][i].get('action')
        targets = torch.Tensor(target[0].shape[0],target[0].shape[1],target[0].shape[2])
        torch.cat(target, out=targets)
        ccs=torch.Tensor(inp[0].shape[0],inp[0].shape[1],inp[0].shape[2])
        torch.cat(inp, out=ccs)
        qvals= target_net(targets)
        actions=actions.type('torch.LongTensor')
        actions=actions.reshape(8,1)
        inps=policy(ccs).gather(1,actions)
        p1,p2=qvals.detach().max(1)
        targ = torch.Tensor(1,p1.shape[0])   
        for num in range(start,start+8,1):
            if num==len(store[ep])-1:
                targ[0][num-start]=rew[0][num-start] 
            else:
                targ[0][num-start]=rew[0][num-start]+gamma*p1[num-start]
        optimizer.zero_grad()
        inps=inps.reshape(1,8)
        loss = criterion(inps,targ)
        loss.backward()
        for param in policy.parameters():
            param.grad.data.clamp(-1,1)
        optimizer.step()
            

In [139]:
def trainDQN(episodes):
    steps_done=0
    avg=0
    for i in range(0,episodes,1):
        if i%100==0 and i!=0:
            print("Episode",i,end=" ")
        env.reset()
        queue=collections.deque()
        for j in range(4):
            state,_,_,_=env.step(random.randint(0,2))
            queue.append(state)
        prev = StackStates(queue)
        prev=torch.from_numpy(prev)
        prev=prev.reshape(1,1,8)
        prev=prev.type('torch.FloatTensor')
        done=False
        steps=0
        rew=0
        moves=0
        while moves<200:
            moves+=1
            eps_threshold = EPS_END + (EPS_START - EPS_END) * \
            math.exp(-1. * steps_done / EPS_DECAY)
            steps+=1
            output=policy(prev)
            action=(output.argmax()).item()
            rand= random.uniform(0,1)
            if rand < eps_threshold:
                action=random.randint(0,2)
            x=env.step(action)
            sc,reward,done,_=x
            avg=avg+reward
            queue.popleft()
            queue.append(sc)
            sc = StackStates(queue)
            sc=torch.from_numpy(sc)
            sc=sc.reshape(1,1,8)
            sc=sc.type('torch.FloatTensor')
            rew=rew+reward
            addEpisode(i,prev.unsqueeze(0),sc.unsqueeze(0),reward,action)
            trainNet(i)
            prev=sc
            steps_done+=1
        terminal = torch.zeros(prev.shape[0],prev.shape[1],prev.shape[2])
        addEpisode(i,prev.unsqueeze(0),terminal.unsqueeze(0),-10,action)
        if i%10==0:
            target_net.load_state_dict(policy.state_dict())
        if i%100==0 and i!=0:
            print(" -> Average reward over last 100 episodes " ,avg/100)
            avg=0

In [140]:
trainDQN(2000)

Episode 100  -> Average reward over last 100 episodes  -202.0
Episode 200  -> Average reward over last 100 episodes  -200.0
Episode 300  -> Average reward over last 100 episodes  -200.0
Episode 400  -> Average reward over last 100 episodes  -200.0
Episode 500  -> Average reward over last 100 episodes  -200.0
Episode 600  -> Average reward over last 100 episodes  -200.0
Episode 700  -> Average reward over last 100 episodes  -200.0
Episode 800  -> Average reward over last 100 episodes  -200.0
Episode 900  -> Average reward over last 100 episodes  -200.0
Episode 1000  -> Average reward over last 100 episodes  -200.0
Episode 1100  -> Average reward over last 100 episodes  -200.0
Episode 1200  -> Average reward over last 100 episodes  -200.0
Episode 1300  -> Average reward over last 100 episodes  -200.0
Episode 1400  -> Average reward over last 100 episodes  -200.0
Episode 1500  -> Average reward over last 100 episodes  -200.0
Episode 1600  -> Average reward over last 100 episodes  -200.0
E